In [1]:
import numpy as np
import matplotlib.pyplot
import seaborn as sns

sns.set()

Создаем верхнетреугольную матрицу, у которой на главной диагонали единицы, а в верхнем треугольнике -1 и такой вектор b, чтобы решением системы был единичный вектор.

In [2]:
def create_j_matrix(n=10):
    A = np.full((n, n), fill_value=-1)
    for i in range(n):
        for j in range(n):
            if i == j: 
                A[i][j] = 1
            if i > j:
                A[i][j] = 0
    
    return A, A @ np.ones(n)

Матрица кажется хорошей, но на самом деле, она очень плохая (обусловленность кошмарная). Тем не менее, питон прекрасно с ней справляется (питон умный).

In [5]:
n = 1000
A, b = create_j_matrix(n)
np.linalg.solve(A, b)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [6]:
A

array([[ 1, -1, -1, ..., -1, -1, -1],
       [ 0,  1, -1, ..., -1, -1, -1],
       [ 0,  0,  1, ..., -1, -1, -1],
       ...,
       [ 0,  0,  0, ...,  1, -1, -1],
       [ 0,  0,  0, ...,  0,  1, -1],
       [ 0,  0,  0, ...,  0,  0,  1]])

А теперь немного возмутим правую часть - сдвинем одно значение на 1е-14 - это почти машинная точность! Видим, что даже умный питон, даже на размерности 90 выдает в ответе какой-то ужасный вектор вместо единичного. Заметим, что невязка очень маленькая даже с учетом того, что ответ совсем не похож на тот, что мы предполагали.

In [7]:
n = 90
A, b = create_j_matrix(n)
b[-1] += 1e-14
x = np.linalg.solve(A, b)

print(x)
print(np.mean(A @ x - b))

[3.08604580e+12 1.54302290e+12 7.71511450e+11 3.85755725e+11
 1.92877862e+11 9.64389312e+10 4.82194656e+10 2.41097328e+10
 1.20548664e+10 6.02743320e+09 3.01371660e+09 1.50685830e+09
 7.53429151e+08 3.76714576e+08 1.88357288e+08 9.41786447e+07
 4.70893229e+07 2.35446619e+07 1.17723315e+07 5.88616623e+06
 2.94308362e+06 1.47154231e+06 7.35771654e+05 3.67886327e+05
 1.83943664e+05 9.19723318e+04 4.59866659e+04 2.29938329e+04
 1.14974165e+04 5.74920824e+03 2.87510412e+03 1.43805206e+03
 7.19526030e+02 3.60263015e+02 1.80631507e+02 9.08157537e+01
 4.59078768e+01 2.34539384e+01 1.22269692e+01 6.61348461e+00
 3.80674230e+00 2.40337115e+00 1.70168558e+00 1.35084279e+00
 1.17542139e+00 1.08771070e+00 1.04385535e+00 1.02192767e+00
 1.01096384e+00 1.00548192e+00 1.00274096e+00 1.00137048e+00
 1.00068524e+00 1.00034262e+00 1.00017131e+00 1.00008565e+00
 1.00004283e+00 1.00002141e+00 1.00001071e+00 1.00000535e+00
 1.00000268e+00 1.00000134e+00 1.00000067e+00 1.00000033e+00
 1.00000017e+00 1.000000

Попробуем решить эту систему с помощью SVD-разложения: сначала найдем это разложение, а потом уберем одно собственное значение.

$$
A = U D V^T \Rightarrow \tilde A = U \tilde D V^T
$$
$$
A x = b, \quad \operatorname{rk} A = n, \quad \sigma _1 \geq \sigma _2 \geq \ldots \geq \sigma _n := 0 
$$
$$
x = V_{n-1} D^{-1}_{n-1} U^T_{n-1} b
$$

In [ ]:
n = 90
A, b = create_j_matrix(n)
b[-1] += 1e-14

U, D, VT = np.linalg.svd(A)

D_inv = np.diag(1. / D[:-1])

x = VT.T[:, :-1] @ D_inv @ U[:, :-1].T @ b

print("Ответ (правильный - единичный вектор): \n", x)
print("Ошибка: ", np.mean(x - np.ones(len(x))))
print("Невязка: ", np.mean(A @ x - b))

Ответ (правильный - единичный вектор): 
 [-0.5         0.25        0.625       0.8125      0.90625     0.953125
  0.9765625   0.98828125  0.99414063  0.99707031  0.99853516  0.99926758
  0.99963379  0.99981689  0.99990845  0.99995422  0.99997711  0.99998856
  0.99999428  0.99999714  0.99999857  0.99999928  0.99999964  0.99999982
  0.99999991  0.99999996  0.99999998  0.99999999  0.99999999  1.
  1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.
  1.          1.   

In [9]:
D

array([5.63595359e+01, 1.88362656e+01, 1.13612234e+01, 8.17844857e+00,
       6.42604991e+00, 5.32342674e+00, 4.57028802e+00, 4.02645702e+00,
       3.61769633e+00, 3.30101144e+00, 3.04978019e+00, 2.84663985e+00,
       2.67978765e+00, 2.54092461e+00, 2.42404960e+00, 2.32471947e+00,
       2.23957790e+00, 2.16604528e+00, 2.10210896e+00, 2.04617779e+00,
       1.99697911e+00, 1.95348436e+00, 1.91485449e+00, 1.88039928e+00,
       1.84954656e+00, 1.82181870e+00, 1.79681447e+00, 1.77419475e+00,
       1.75367142e+00, 1.73499837e+00, 1.71796445e+00, 1.70238762e+00,
       1.68811035e+00, 1.67499573e+00, 1.66292437e+00, 1.65179178e+00,
       1.64150619e+00, 1.63198678e+00, 1.62316213e+00, 1.61496899e+00,
       1.60735113e+00, 1.60025851e+00, 1.59364641e+00, 1.58747484e+00,
       1.58170792e+00, 1.57631340e+00, 1.57126222e+00, 1.56652819e+00,
       1.56208760e+00, 1.55791899e+00, 1.55400288e+00, 1.55032158e+00,
       1.54685900e+00, 1.54360046e+00, 1.54053255e+00, 1.53764306e+00,
      

Невязка стала больше, но такой ответ гораздо больше похож на тот, что мы ожидали.

In [30]:
def create_g_matrix(n=10):
    A = np.full((n, n), fill_value=-1.)
    for i in range(n):
        for j in range(n):
            # print(1. / float(i + j + 1))
            A[i][j] = 1. / float(i + j + 1)

    
    return A, A @ np.ones(n)

In [31]:
n = 10
A, b = create_g_matrix(n)
A

array([[1.        , 0.5       , 0.33333333, 0.25      , 0.2       ,
        0.16666667, 0.14285714, 0.125     , 0.11111111, 0.1       ],
       [0.5       , 0.33333333, 0.25      , 0.2       , 0.16666667,
        0.14285714, 0.125     , 0.11111111, 0.1       , 0.09090909],
       [0.33333333, 0.25      , 0.2       , 0.16666667, 0.14285714,
        0.125     , 0.11111111, 0.1       , 0.09090909, 0.08333333],
       [0.25      , 0.2       , 0.16666667, 0.14285714, 0.125     ,
        0.11111111, 0.1       , 0.09090909, 0.08333333, 0.07692308],
       [0.2       , 0.16666667, 0.14285714, 0.125     , 0.11111111,
        0.1       , 0.09090909, 0.08333333, 0.07692308, 0.07142857],
       [0.16666667, 0.14285714, 0.125     , 0.11111111, 0.1       ,
        0.09090909, 0.08333333, 0.07692308, 0.07142857, 0.06666667],
       [0.14285714, 0.125     , 0.11111111, 0.1       , 0.09090909,
        0.08333333, 0.07692308, 0.07142857, 0.06666667, 0.0625    ],
       [0.125     , 0.11111111, 0.1      

Попробуем то же самое для извесной своей ужасностью матрицы Гильберта.

In [34]:
n = 25
A, b = create_g_matrix(n)
np.linalg.solve(A, b)

array([   0.99999924,    1.00011425,    0.99594795,    1.05775147,
          0.61856645,    1.9824775 ,    2.59586386,  -15.83659106,
         42.51805794,  -35.14969359,   -3.62654458,  -25.86940942,
        142.13946664, -114.33642281,  -33.08823714,   -0.86302566,
         92.60997782,   37.51409768, -101.7008774 ,   -0.35543787,
        -11.45556857,   62.01584847,   13.47687558,  -52.63649857,
         20.39326297])

Питон справился... Человечеству осталось недолго. Но это потому что он умный, любой обычный метод сломается уже на размерности 20.

In [36]:
n = 25
A, b = create_g_matrix(n)
# b[-1] += 1e-14
x = np.linalg.solve(A, b)

print(x)
print(np.mean(A @ x - b))

[   0.99999924    1.00011425    0.99594795    1.05775147    0.61856645
    1.9824775     2.59586386  -15.83659106   42.51805794  -35.14969359
   -3.62654458  -25.86940942  142.13946664 -114.33642281  -33.08823714
   -0.86302566   92.60997782   37.51409768 -101.7008774    -0.35543787
  -11.45556857   62.01584847   13.47687558  -52.63649857   20.39326297]
5.329070518200751e-16


А тут питон уже не справляется: не доволен матрицей, хотя мы-то знаем, что матрица не сингулярная. Впрочем, наш метод тоже не справился...

In [40]:
n = 25
A, b = create_g_matrix(n)
# b[-1] += 1e-14

U, D, VT = np.linalg.svd(A)

D_inv = np.diag(1. / D[:-10])

x = VT.T[:, :-10] @ D_inv @ U[:, :-10].T @ b

print("Ответ (правильный - единичный вектор): \n", x)
print("Ошибка: ", np.mean(x - np.ones(len(x))))
print("Невязка: ", np.mean(A @ x - b))

Ответ (правильный - единичный вектор): 
 [0.99999997 0.99999809 0.99972534 1.00292969 0.98828125 1.046875
 0.75       1.3125     1.04101562 0.78125    0.9921875  1.140625
 0.953125   1.01953125 0.84375    0.875      0.9921875  0.890625
 1.078125   1.01672363 0.875      0.84375    0.9765625  0.8125
 1.09375   ]
Ошибка:  -0.026959305852651595
Невязка:  -0.023376350370686427


In [38]:
D

array([1.95175652e+00, 5.34124132e-01, 9.15587547e-02, 1.22685349e-02,
       1.37443087e-03, 1.32008752e-04, 1.10125336e-05, 8.04060040e-07,
       5.16143770e-08, 2.92004527e-09, 1.45716228e-10, 6.41062744e-12,
       2.48192983e-13, 8.43158331e-15, 2.48370922e-16, 1.53223482e-17,
       1.18485153e-17, 8.68373407e-18, 7.65927055e-18, 7.35202463e-18,
       6.37921647e-18, 4.35542949e-18, 3.67491562e-18, 1.58799492e-18,
       5.08320682e-19])

In [16]:
D

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])